In [30]:
import json, openai #main players
import configparser #security
import warnings #support
warnings.filterwarnings('ignore')
import uuid #prompt
from time import sleep #give gpt some break
import requests #get data from api end point

In [2]:
credentials = configparser.ConfigParser()
credentials.read_file(open('apidata.config'))

In [3]:
my_key = credentials['OPENAI']['KEY']
my_org = credentials['OPENAI']['org']

In [4]:
#Lets begin

In [31]:
places = ['India','United States of America',
         'Pakistan','Russia']
year = [2022, 2024, 2026, 2028]
ideas1 = ['Power','Ability','automation','danger']
ideas2 = ['Government','education','society','space faring']

In [6]:
str(uuid.uuid4())

'69b6c1c3-ddd1-4a66-8d89-66977ada2bae'

In [7]:
prompt_list = []
for plc in places:
    for yr in year:
        for id1 in ideas1:
            for id2 in ideas2:
                prompt =f"""{uuid.uuid4()}
Imagine you are the highest earning youtube influencer.
The year is {yr} and you are living in {plc}.You are asked to influence the 
masses on topic {id1}. Write 5 points each 100 words elaborating {id1} impact on {id2}
so it can be made in to a video"""
                prompt_list.append(prompt)

In [8]:
prompt_parts = []
for plc in places:
    for yr in year:
        for id1 in ideas1:
            for id2 in ideas2:
                prompt_part =f"""Year: {yr}\nPlace: {plc}\nIdea1:{id1}\nIdea2:{id2}"""
                prompt_parts.append(prompt_part)

In [32]:
len(prompt_list)

256

In [33]:
len(prompt_parts)

256

In [11]:
prompt_list[0:10]

['d16eba68-aefd-4895-985f-3d15a5b641d5\nImagine you are the highest earning youtube influencer.\nThe year is 2022 and you are living in India.You are asked to influence the \nmasses on topic Power. Write 5 points each 100 words elaborating Power impact on Government\nso it can be made in to a video',
 '46c63efc-6887-4a4c-b8c1-e20007cbfeba\nImagine you are the highest earning youtube influencer.\nThe year is 2022 and you are living in India.You are asked to influence the \nmasses on topic Power. Write 5 points each 100 words elaborating Power impact on education\nso it can be made in to a video',
 '2d98b410-727f-48b9-acc2-25baae57fbbb\nImagine you are the highest earning youtube influencer.\nThe year is 2022 and you are living in India.You are asked to influence the \nmasses on topic Power. Write 5 points each 100 words elaborating Power impact on society\nso it can be made in to a video',
 '9212d14a-6a53-4ba2-a3e0-c2d66b55c5fa\nImagine you are the highest earning youtube influencer.\nT

In [12]:
prompt_parts[0:10]

['Year: 2022\nPlace: India\nIdea1:Power\nIdea2:Government',
 'Year: 2022\nPlace: India\nIdea1:Power\nIdea2:education',
 'Year: 2022\nPlace: India\nIdea1:Power\nIdea2:society',
 'Year: 2022\nPlace: India\nIdea1:Power\nIdea2:space faring',
 'Year: 2022\nPlace: India\nIdea1:Ability\nIdea2:Government',
 'Year: 2022\nPlace: India\nIdea1:Ability\nIdea2:education',
 'Year: 2022\nPlace: India\nIdea1:Ability\nIdea2:society',
 'Year: 2022\nPlace: India\nIdea1:Ability\nIdea2:space faring',
 'Year: 2022\nPlace: India\nIdea1:automation\nIdea2:Government',
 'Year: 2022\nPlace: India\nIdea1:automation\nIdea2:education']

In [13]:
str(uuid.uuid1()).split('-')[0]

'0ff45c4c'

In [14]:
def gpt3_completion(org, key,prompt,prompt_part,
                    engine='text-davinci-002',temp=1.0,
                   top_p=1.0,tokens=500,freq_pen=0.0,
                   pres_pen=0.0,stop=['asdfasdf','asdfasdf'],
                   filename='temp_file.jsonl'):
    """The function will take the prompt and the prompt parts to 
    begin the data collection. Once the data collection is completed, 
    the prompt part will be used for fine tuning. The prompt used will 
    be discarded"""
    openai.api_key = key
    openai.organization = org
    max_retry=5
    retry = 0
    prompt = prompt.encode(encoding='ASCII',errors='ignore').decode()
    while True:
        try:
            response = openai.Completion.create(
            model=engine,
            prompt=prompt,
            temperature=temp,
            max_tokens=tokens,
            top_p=top_p,
            frequency_penalty=freq_pen,
            presence_penalty=pres_pen,
            stop=stop)
            
            text = response['choices'][0]['text'].strip()
            outprompt = prompt_part
            dict_reply = {'prompt':outprompt,'completion':text}
            with open(filename,'a+') as taker:
                json.dump(dict_reply,fp=taker)
                taker.write('\n')
            
            return dict_reply
        except Exception as e:
            retry += 1
            if retry > max_retry:
                return f'GPT errored out {e}'
            sleep(1)

In [15]:
dict_to_work = gpt3_completion(org=my_org,key=my_key,
                prompt=prompt_list[0],
                prompt_part=prompt_parts[0],
               filename='tutorial_payload.jsonl')

In [34]:
prompt_list[0]

'd16eba68-aefd-4895-985f-3d15a5b641d5\nImagine you are the highest earning youtube influencer.\nThe year is 2022 and you are living in India.You are asked to influence the \nmasses on topic Power. Write 5 points each 100 words elaborating Power impact on Government\nso it can be made in to a video'

In [16]:
dict_to_work


{'prompt': 'Year: 2022\nPlace: India\nIdea1:Power\nIdea2:Government',
 'completion': 'by you.\n\n\n1. Power has a huge impact on the government.\n\n2. It can be used to make the government more efficient and effective.\n\n3. It can also be used to corrupt the government and make it less effective.\n\n4. Power can be used to make the government more accountable to the people.\n\n5. Power can also be used to make the government more responsive to the needs of the people.'}

In [17]:
!cat tutorial_payload.jsonl

{"prompt": "Year: 2022\nPlace: India\nIdea1:Power\nIdea2:Government", "completion": "by you.\n\n\n1. Power has a huge impact on the government.\n\n2. It can be used to make the government more efficient and effective.\n\n3. It can also be used to corrupt the government and make it less effective.\n\n4. Power can be used to make the government more accountable to the people.\n\n5. Power can also be used to make the government more responsive to the needs of the people."}


In [24]:
for ind,pmt in enumerate(prompt_list[0:5]):
    #print(prompt_list[ind])
    gpt3_completion(org=my_org,key=my_key,
                prompt=prompt_list[ind],
                   prompt_part=prompt_parts[ind],
                    filename='tutorial_payload.jsonl')

In [25]:
!cat tutorial_payload.jsonl

{"prompt": "Year: 2022\nPlace: India\nIdea1:Power\nIdea2:Government", "completion": "by you.\n\n\n1. Power has a huge impact on the government.\n\n2. It can be used to make the government more efficient and effective.\n\n3. It can also be used to corrupt the government and make it less effective.\n\n4. Power can be used to make the government more accountable to the people.\n\n5. Power can also be used to make the government more responsive to the needs of the people."}
{"prompt": "Year: 2022\nPlace: India\nIdea1:Power\nIdea2:Government", "completion": "ad.\n\n1. Power has the ability to make or break a government.\n\n2. It can either be used to support a government or to overthrow it.\n\n3. In the hands of the wrong people, power can be extremely dangerous.\n\n4. It is important to use power wisely and for the benefit of all.\n\n5. With great power comes great responsibility."}
{"prompt": "Year: 2022\nPlace: India\nIdea1:Power\nIdea2:education", "completion": ".\n\n1. Power can be a g

In [18]:
def file_upload(filename, purpose='fine-tune'):
    resp = openai.File.create(purpose=purpose,file=open(file=filename,
                                                        mode='rb'))
    print(resp)
    return resp

In [19]:
def file_list():
    #print(openai.File.list())
    return openai.File.list()

In [20]:
def finetune_model(fileId, suffix,model='davinci',api_key=my_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    payload = {'training_file':fileId,'model':model,'suffix':suffix}
    resp = requests.request(method='POST',
                            url='https://api.openai.com/v1/fine-tunes',
                           headers=header,
                           json=payload,timeout=40)
    print(resp.json())

In [21]:
def finetune_event(ftId,api_key=my_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    resp = requests.request(method='GET',
                url=f'https://api.openai.com/v1/fine-tunes/{ftId}/events',
                           headers=header,timeout=40)
    print(resp.json())

In [22]:
def finetune_get(ftId,api_key=my_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    resp = requests.request(method='GET',
                url=f'https://api.openai.com/v1/fine-tunes/{ftId}',
                           headers=header,timeout=40)
    print(resp.json())

In [26]:
file_upload(filename='tutorial_payload.jsonl')

{
  "bytes": 2661,
  "created_at": 1678007422,
  "filename": "file",
  "id": "file-uBoRaXeiUIHYgmjOzbo8KBSc",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


<File file id=file-uBoRaXeiUIHYgmjOzbo8KBSc at 0x7f906b92d080> JSON: {
  "bytes": 2661,
  "created_at": 1678007422,
  "filename": "file",
  "id": "file-uBoRaXeiUIHYgmjOzbo8KBSc",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [27]:
finetune_model(fileId='file-uBoRaXeiUIHYgmjOzbo8KBSc',
               suffix='tutorial_generator',api_key=my_key)

{'object': 'fine-tune', 'id': 'ft-zyks6ovSBOrRwVhcdhidziax', 'hyperparams': {'n_epochs': 4, 'batch_size': None, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': None}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-uBoRaXeiUIHYgmjOzbo8KBSc', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 2661, 'created_at': 1678007422, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [], 'created_at': 1678007437, 'updated_at': 1678007437, 'status': 'pending', 'fine_tuned_model': None, 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-zyks6ovSBOrRwVhcdhidziax', 'created_at': 1678007437}]}


In [28]:
finetune_event(ftId='ft-zyks6ovSBOrRwVhcdhidziax',api_key=my_key)

{'object': 'list', 'data': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-zyks6ovSBOrRwVhcdhidziax', 'created_at': 1678007437}]}


In [29]:
finetune_get(ftId='ft-iUuWQgOAPtb31KnOmo8emkiv',api_key=my_key)

{'object': 'fine-tune', 'id': 'ft-iUuWQgOAPtb31KnOmo8emkiv', 'hyperparams': {'n_epochs': 4, 'batch_size': 1, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': 0.1}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-16SdoYlLlW2IRQEykCWwjvWv', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 6195, 'created_at': 1677993663, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [{'object': 'file', 'id': 'file-DTl0Zp5T4SX4AkixZphY1Qyo', 'purpose': 'fine-tune-results', 'filename': 'compiled_results.csv', 'bytes': 1615, 'created_at': 1677995208, 'status': 'processed', 'status_details': None}], 'created_at': 1677994822, 'updated_at': 1677995208, 'status': 'succeeded', 'fine_tuned_model': 'davinci:ft-kamal:influencer-generator-2023-03-05-05-46-47', 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-iUuWQgOAPtb31KnOmo8emkiv', 'created_at': 1

In [36]:
reply_dict_holder = []
for ind,pmt in enumerate(prompt_list[:5]):
    #print(prompt_list[ind])
    reply_dict = gpt3_completion(org=my_org,key=my_key,
                prompt=prompt_list[ind],
                   prompt_part=prompt_parts[ind],
                    filename='full_model_payload.jsonl')
    reply_dict_holder.append(reply_dict)

In [37]:
len(reply_dict_holder)

5

In [38]:
import pandas as pd

In [39]:
reply_dataframe = pd.DataFrame(reply_dict_holder)
reply_dataframe

,prompt,completion
0,Year: 2022\nPlace: India\nIdea1:Power\nIdea2:G...,.\n\n1. India is a rapidly developing country ...
1,Year: 2022\nPlace: India\nIdea1:Power\nIdea2:e...,content\n\n1. Power can be a great motivator f...
2,Year: 2022\nPlace: India\nIdea1:Power\nIdea2:s...,.\n\n\n1.Power can be used to make a real diff...
3,Year: 2022\nPlace: India\nIdea1:Power\nIdea2:s...,.\n\n1. Power is the ability to influence or c...
4,Year: 2022\nPlace: India\nIdea1:Ability\nIdea2...,by you which can be a script for your video.\n...


In [40]:
reply_dataframe['completion_length'] = reply_dataframe.completion. \
                            apply(lambda x: len(x))
reply_dataframe

,prompt,completion,completion_length
0,Year: 2022\nPlace: India\nIdea1:Power\nIdea2:G...,.\n\n1. India is a rapidly developing country ...,446
1,Year: 2022\nPlace: India\nIdea1:Power\nIdea2:e...,content\n\n1. Power can be a great motivator f...,409
2,Year: 2022\nPlace: India\nIdea1:Power\nIdea2:s...,.\n\n\n1.Power can be used to make a real diff...,322
3,Year: 2022\nPlace: India\nIdea1:Power\nIdea2:s...,.\n\n1. Power is the ability to influence or c...,283
4,Year: 2022\nPlace: India\nIdea1:Ability\nIdea2...,by you which can be a script for your video.\n...,1481


In [41]:
reply_dataframe['completion_length'] = reply_dataframe.completion. \
                            apply(lambda x: len(x))
final_df = reply_dataframe[reply_dataframe.completion_length > 375]

In [42]:
final_df

,prompt,completion,completion_length
0,Year: 2022\nPlace: India\nIdea1:Power\nIdea2:G...,.\n\n1. India is a rapidly developing country ...,446
1,Year: 2022\nPlace: India\nIdea1:Power\nIdea2:e...,content\n\n1. Power can be a great motivator f...,409
4,Year: 2022\nPlace: India\nIdea1:Ability\nIdea2...,by you which can be a script for your video.\n...,1481


In [141]:
final_df.drop('completion_length',inplace=True,axis=1)

In [151]:
with open('filtered_file.jsonl','w') as jso:
    for d in final_df.to_dict(orient='records'):
        json.dump(obj=d,fp=jso)
        jso.write('\n')

In [152]:
!cat filtered_file.jsonl

{"prompt": "Year: 2022\nPlace: India\nIdea1:Power\nIdea2:Government", "completion": "by a production house.\n\n1. In a democracy, power ultimately lies with the people. The government is responsible for ensuring that the will of the people is carried out and that their rights are protected.\n\n2. Government has the power to make laws and pass policies that can have a huge impact on the lives of its citizens.\n\n3. Government also has the power to enforce its laws and policies, and to use force if necessary.\n\n4. Government has the power to tax its citizens and to spend that money as it sees fit.\n\n5. Government has the power to borrow money, and to make decisions about how that money is to be used."}
{"prompt": "Year: 2022\nPlace: India\nIdea1:Power\nIdea2:education", "completion": "(1-2 minutes)\n\n1. In today's world, power is everything. It can make or break you.\n\n2. Power is what gives you the ability to make things happen.\n\n3. Education is the key to success. It opens doors 

In [153]:
file_upload(filename='filtered_file.jsonl')

{
  "bytes": 147793,
  "created_at": 1678000239,
  "filename": "file",
  "id": "file-r9vI0KULMmBUYOvTyEEmQb2j",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


<File file id=file-r9vI0KULMmBUYOvTyEEmQb2j at 0x7f56d6cf6160> JSON: {
  "bytes": 147793,
  "created_at": 1678000239,
  "filename": "file",
  "id": "file-r9vI0KULMmBUYOvTyEEmQb2j",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [155]:
finetune_model(fileId='file-r9vI0KULMmBUYOvTyEEmQb2j',
               suffix='new_video_gen_model')

{'object': 'fine-tune', 'id': 'ft-7DGxEQjWAbRJlfCZrt3wQQwn', 'hyperparams': {'n_epochs': 4, 'batch_size': None, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': None}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-r9vI0KULMmBUYOvTyEEmQb2j', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 147793, 'created_at': 1678000239, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [], 'created_at': 1678000327, 'updated_at': 1678000327, 'status': 'pending', 'fine_tuned_model': None, 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-7DGxEQjWAbRJlfCZrt3wQQwn', 'created_at': 1678000327}]}
